In [1]:
import numpy as np
import pandas as pd
import random
import pickle
import itertools
import folium #mapping
import seaborn as sns #color palette
#from colormap import rgb2hex #chnage RGB codes to hex codes

In [2]:
## USE THIS CODE TO IMPORT DATA

file_name = 'data/results/results_toydata.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

##[(supplier_lat, supplier_lon), (receiver_lat, receiver_lon), distance, volume_transferred]
#loaded_list[0:5]

In [3]:
#convert to dataframe

df = pd.DataFrame(loaded_list, columns = ['supplier', 'receiver', 'distance', 'volume'])
df.head()
#df.shape #109031, 4

#list-ify the suppliers to get rid of weird numpy object datatype
supplier_split = df.supplier.to_list()
receiver_split = df.receiver.to_list()
#supplier_split[0:5]


#convert suppliers to lat and lon
supplier_lat= [supplier_split[j][0] for j in range(0,len(supplier_split))]
supplier_lon = [supplier_split[j][1] for j in range(0,len(supplier_split))]

#convert to list of receiver lat and long
receiver_lat= [receiver_split[j][0] for j in range(0,len(receiver_split))]
receiver_lon = [receiver_split[j][1] for j in range(0,len(receiver_split))]

#add back into the df
df['supplier_lat'] = supplier_lat
df['supplier_lon'] = supplier_lon
df['receiver_lat'] = receiver_lat
df['receiver_lon'] = receiver_lat

df.head()

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[36.92375, -120.10412]","[36.23867035, -120.38221741]",49.705671,363.79758,36.92375,-120.10412,36.238670,36.238670
1,"[36.92375, -120.10412]","[36.24571609, -120.33669281]",48.508588,955.40343,36.92375,-120.10412,36.245716,36.245716
2,"[36.92375, -120.10412]","[36.25873947, -120.32772064]",47.510457,818.13738,36.92375,-120.10412,36.258739,36.258739
3,"[36.92375, -120.10412]","[36.224933, -120.303681]",49.447437,240.09383,36.92375,-120.10412,36.224933,36.224933
4,"[36.92375, -120.10412]","[36.255945, -120.34494]",47.954999,158.00688,36.92375,-120.10412,36.255945,36.255945


In [4]:
#subset data for initial map
df_small = df.head(1000)

In [5]:

# #create a color paltte
# palette = sns.color_palette(None, 10)
# palette

# palette[0]
# palette = palette.as_hex()
# palette[0]
# palette = [i.replace('#','') for i in palette]
# palette

#color palette
palette = ['red', 'pink', 'gray','green', 'purple', 'orange',  'darkblue'] #, 
 #          ‘darkgreen’, ‘cadetblue’, ‘darkpurple’, ‘white’, ‘pink’, ‘lightblue’, ‘lightgreen’, 
  #         ‘gray’, ‘black’, ‘lightgray’] 'darkred','lightred','blue','beige','white',

palette

['red', 'pink', 'gray', 'green', 'purple', 'orange', 'darkblue']

In [6]:
df_small

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[36.92375, -120.10412]","[36.23867035, -120.38221741]",49.705671,363.79758,36.92375,-120.10412,36.238670,36.238670
1,"[36.92375, -120.10412]","[36.24571609, -120.33669281]",48.508588,955.40343,36.92375,-120.10412,36.245716,36.245716
2,"[36.92375, -120.10412]","[36.25873947, -120.32772064]",47.510457,818.13738,36.92375,-120.10412,36.258739,36.258739
3,"[36.92375, -120.10412]","[36.224933, -120.303681]",49.447437,240.09383,36.92375,-120.10412,36.224933,36.224933
4,"[36.92375, -120.10412]","[36.255945, -120.34494]",47.954999,158.00688,36.92375,-120.10412,36.255945,36.255945
...,...,...,...,...,...,...,...,...
995,"[37.4166, -122.03599]","[35.26726913, -119.34642029]",210.863250,560.24732,37.41660,-122.03599,35.267269,35.267269
996,"[37.4166, -122.03599]","[35.29407501, -119.52223969]",202.649803,325.88290,37.41660,-122.03599,35.294075,35.294075
997,"[37.4166, -122.03599]","[35.2630043, -119.45105743]",206.966271,306.94616,37.41660,-122.03599,35.263004,35.263004
998,"[37.4166, -122.03599]","[35.2785759, -119.43296814]",206.894146,766.79124,37.41660,-122.03599,35.278576,35.278576


In [11]:
#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers
unique_suppliers['supplier_lon'][0]

-120.10412

In [29]:
mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=20 )
receiver_count = 0
 
#plot all the suppliers 
latlon = df_small['supplier']
mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=5 )
#for coord in latlon:
#    folium.CircleMarker( location=[ coord[0], coord[1] ], fill=True,color='black', fill_color='black',radius=5 ).add_to( mapit ) #'

#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers

#loop through suppliers, paint all the receivers in the same color
for i in range(0,(unique_suppliers.shape[0])):
    
    #big circle for supplier
    folium.CircleMarker( location=[ unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i] ], fill=True, fill_color=palette[i], color=palette[i],  radius=5 ).add_to( mapit ) #'
    
    #find all receivers associated with the supplier late and lon, filter the df
    this_lat = unique_suppliers['supplier_lat'][i]
    this_lon = unique_suppliers['supplier_lon'][i]
    receivers = df_small[(df_small['supplier_lat'] == this_lat) & (df_small['supplier_lon'] == this_lon)]
    
    #plot the receivers with a new color each time
    latlon = receivers['receiver']
    #print(receivers.shape[0])
    #print(receivers.shape[1])
    receiver_count += receivers.shape[0] #verify 1000
    
    #for each receiver  
    for coord in latlon: 
        
        #line coordinates
        line_coords = [ [unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i]],
                        [coord[0], coord[1] ]  ]
        my_PolyLine= folium.PolyLine(locations=line_coords,color=palette[i],weight=1)
        mapit.add_child(my_PolyLine)
        
        #dot for the receiver
        folium.CircleMarker( location=[ coord[0], coord[1] ], fill=True, fill_color=palette[i], color=palette[i],  radius=1 ).add_to( mapit )

print(receiver_count)
mapit

1000


In [16]:
# #latlon = records[0:1000] #[ (51.249443914705175, -0.13878830247011467), (51.249443914705175, -0.13878830247011467), (51.249768239976866, -2.8610415615063034)]
# latlon = df_small['receiver']
# mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=6 )
# for coord in latlon:
#     folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=8 ).add_to( mapit )
    
# mapit

In [26]:
import folium

# Coordinates are 10 points on the great circle from Boston to
# San Francisco.
# Reference: http://williams.best.vwh.net/avform.htm#Intermediate
#coordinates = [
   # [42.3581, -71.0636],
    #[42.82995815, -74.78991444],
    #[43.17929819, -78.56603306],
    #[43.40320216, -82.37774519],
    #[43.49975489, -86.20965845],
    #[41.4338549, -108.74485069],
    #[40.67471747, -112.29609954],
    #[39.8093434, -115.76190821],
    #[38.84352776, -119.13665678],
  #  [37.7833, -122.4167]]

line_coords = [ [unique_suppliers['supplier_lat'][0],unique_suppliers['supplier_lon'][0]],
                        [coord[0], coord[1] ]  ]

m.add_child(my_PolyLine)

# Create the map and add the line
m = folium.Map(location=[41.9, -97.3], zoom_start=4)
my_PolyLine=folium.PolyLine(locations=line_coords,weight=5) #coordinates
m.add_child(my_PolyLine)
# m.save('line_example_newer.html')